<a href="https://colab.research.google.com/github/gabrielsimas/rio-viagens-onibus-analytics/blob/feature%2Fcriacao-ambiente/ARTIGO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚌 MVP Engenharia de Dados - O Rio de Janeiro e os Ônibus: Entendendo o Caos!
## ***Nome:*** **Luís Gabriel Nascimento Simas**
## ***Matrícula:*** **4052025000943**

## **Objetivo**

### *O objetivo do trabalho deve ser muito bem detalhado; é um planejamento do trabalho, contendo de forma clara e objetiva o problema a ser resolvido e as perguntas de negócio a serem respondidas. Será avaliada a qualidade desta descrição.*
### *Comece pelo objetivo do seu trabalho. Antes de iniciar sua busca pelos dados, pense e descreva claramente qual problema deseja resolver com este MVP. Enumere as perguntas que deseja responder.*
### *É de extrema importância que esta etapa seja feita antes de iniciar qualquer outra etapa.*
### *Uma vez traçado o objetivo e conhecendo bem qual problema se deseja resolver, quais perguntas se deseja responder, é hora de iniciar a busca pelos dados.*
### *Não é necessário atingir todos os objetivos desenhados nesta seção. Assim, não remova perguntas as quais não se conseguiu responder. Deixe a documentação do objetivo intacta e faça uma boa discussão do atingimento deste ao final do trabalho (vide Autoavaliação).*


### O Rio de Janeiro, mundialmente conhecido por suas belezas naturais, enfrenta um desafio urbano de proporções continentais: a mobilidade pública. O sistema de transporte rodoviário, vital para o funcionamento da metrópole, é frequentemente associado à palavra "**Caos**".
### A população carioca convive diariamente com incertezas: tempos de espera imprevisíveis, frota envelhecida, desconforto térmico em uma cidade tropical e falhas na cobertura do serviço, especialmente em horários noturnos. Embora iniciativas como o BRT (*Bus Rapid Transit*) e o BRS (*Bus Rapid System*) tenham sido implementadas para mitigar a lentidão, a percepção de qualidade do serviço permanece aquém do ideal.
### Não se trata apenas de desconforto, mas de um problema sistêmico que envolve a gestão de consórcios privados, a fiscalização do poder público e o impacto direto na qualidade de vida e produtividade do cidadão.
### Se por um lado o cenário físico é caótico, o cenário digital oferece uma oportunidade de organização e entendimento. A Prefeitura da Cidade do Rio de Janeiro (PCRJ), através do portal *[data.rio](https://data.rio)*, disponibiliza um vasto repositório de dados abertos em vários formatos e alguns até como tabelas no **Google** *BigQuery*.
### Esses dados incluem registros detalhados de viagens (baseados em bilhetagem e telemetria consolidada), cadastro técnico da frota, dados meteorológicos do Alerta Rio e registros de reclamações da central 1746.

### Para guiar o desenvolvimento do pipeline de dados, definimos as seguintes questões-chave que esse trabalho deseja responder.


- ### **Dinâmica de Viagens e Demanda**
    - #### 1. Em quais dias e faixas horárias o volume de viagens realizadas atinge seu pico?
    - #### 2. Existe uma redução sensível na oferta de viagens nos finais de semana comparados aos dias úteis?
    - #### 3. Qual é a duração média das viagens por Consórcio? Existe discrepância significativa entre eles?

- ### **Consistência Operacional**
    - #### 4. O tempo de viagem nas principais linhas é consistente ou apresenta alto desvio padrão (imprevisibilidade)?
    - #### 5. Em dias classificados como "Chuvosos" (precipitação média acima de um limiar), observa-se aumento no tempo médio de deslocamento?
    - #### 6. A chuva influencia na quantidade total de viagens realizadas (redução da oferta ou demanda)?

- ### Perfil da frota e qualidade
    - #### 7. Qual é a idade média da frota de ônibus ativa no Rio de Janeiro?
    - #### 8. Quais consórcios operam com a frota mais moderna e quais possuem a frota mais obsoleta?
    - #### 9. Existe correlação entre a idade da frota e a presença de ar-condicionado funcional?

- ### Percepção do Usuário (1746)
    - #### 10. Quais são as categorias de reclamação mais frequentes relacionadas ao transporte rodoviário?

### **Escopo temporal**
#### A análise compreenderá o período de **01/01/2024** a **19/12/2025**, permitindo uma visão longitudinal que abrange sazonalidades, dias úteis, feriados e variações climáticas ao longo de quase dois anos completos.
#### Antes de começar a explicar o mecanismo de coleta de dados que foi utilizado, é importante mostrar em qual arquitetura nosso trabalho está sendo desenvolvido/implementado.


## 🏛️ **Arquitetura da Solução**: Data Lakehouse Híbrido (VPS + GCP)

### Esta seção detalha os componentes tecnológicos do MVP. A arquitetura adota uma estratégia **Híbrida**, combinando a otimização de recursos de um VPS (Virtual Private Server) com a escalabilidade de armazenamento da Nuvem Pública (Google Cloud).

---

### **1. Provisionamento de Infraestrutura (IaC)**

### A infraestrutura na nuvem não foi criada manualmente. Todo o ambiente (Buckets, Permissões, APIs) foi definido via **Terraform**, garantindo que o ambiente seja reprodutível, auditável e imune a erros humanos.

### 🏗️ **Terraform Workflow**
### **Papel:** Provisionamento de Nuvem.
* ### **Função:** Cria e gerencia os recursos na GCP (Landing, Bronze, Silver, Gold) de forma declarativa.
* ### **Ciclo de Vida:** O deploy seguiu as fases de *Bootstrap*, *Validation*, *Plan* e *Apply*.

<details>

<summary>
<strong>
Clique para expandir: Evidências da Execução do Terraform
</strong>
</summary>
<br>

<h2 style="margin-bottom:0">1. Preparação do Ambiente (Bootstrap)</h2>
<img src="https://drive.google.com/uc?id=1Iq7M2xjMHbjlrEMHYy36VCb8tu1WXJQZ" width="100%" alt="Script Inicial">
<p><em>Setup inicial das variáveis e autenticação no Google Cloud Shell.</em></p>
<hr>

<h2 style="margin-bottom:0">2. Validação e Planejamento (Plan)</h2>
<img src="https://drive.google.com/uc?id=1tdGPpca2xtP4oKW0zmfELg5opni451Ao" width="100%" alt="Terraform Validate">
<p><em>Validando a sintaxe dos arquivos .tf para garantir qualidade do código.</em></p>
<br>
<img src="https://drive.google.com/uc?id=1PIAl4zRuyEZYhEJJVuPNaItXQIO99RUm" width="100%" alt="Terraform Plan">
<p><em>Terraform Plan: O sistema calcula o "estado futuro" e prevê as mudanças antes de tocar na nuvem.</em></p>
<hr>

<h2 style="margin-bottom:0">3. Aplicação e Resultado (Apply)</h2>
<img src="https://drive.google.com/uc?id=1faj6oTVGA2jjHIyjzZ5EzMl1kRyUxE8S" width="100%" alt="Terraform Apply Sucesso">
<p><em>Terraform Apply: Criação automatizada dos recursos (Buckets, Service Accounts).</em></p>
<br>
<img src="https://drive.google.com/uc?id=1JaqcI2S5XlZiu6WKHt6Ibn3nzoiGHKzK">
<p><em>Resultado Final: O ambiente pronto no console da Google Cloud, refletindo exatamente o código.</em></p>

</details>

---

## **2. Servidor de Processamento (Compute On-Premise)**

Para viabilizar o uso de ferramentas corporativas (Dremio/Java) em um hardware limitado, foi realizado um trabalho avançado de **Engenharia de Sistemas** no VPS.

### 🖥️ **Especificações e Otimização**
* **Hardware:** VPS com processador AMD EPYC (4 vCPUs) e **6GB de RAM**.
* **Estratégia de Swap:** Devido à limitação de memória física, configurou-se uma área de swap de **22GB+** via terminal Linux. Isso permite que o *Heap* do Java (Dremio) e os processos do Airflow rodem simultaneamente sem causar *Out Of Memory (OOM)*.

### 🐧 **Hardening e Segurança**
O ambiente segue o princípio do menor privilégio, evitando a execução de containers como `root` e utilizando chaves SSH para acesso.

<details>
<summary><strong>🎥 Clique para expandir: Engenharia de Sistemas (Linux Tuning)</strong></summary>
<br>

<h2 style="margin-bottom:0">1. Acesso Seguro e Criação de Usuário</h2>
<img src="https://drive.google.com/uc?id=1KVESwnVCpXmFbDKCJSFuEQzm7-cp6eB4" width="100%" alt="Acesso SSH">
<p><em>Acesso remoto seguro via SSH com chaves criptografadas (sem senha).</em></p>
<br>
<img src="https://drive.google.com/uc?id=1-ruL8NLRbhD9FZ333c7H0A2sVInfK411" width="100%" alt="User Setup">
<p><em>Segurança: Criação de usuário dedicado para evitar execução de containers como root.</em></p>
<hr>

<h2 style="margin-bottom:0">2. Expansão de Memória (Swap Strategy)</h2>
<img src="https://drive.google.com/uc?id=1jy_hFiKoHqKumGM-0ZMaYvaQ8GZdCEdN" width="100%" alt="Swap Creation">
<p><em>Engenharia: Criação manual de arquivo de Swap e ajuste de swappiness para suportar carga pesada.</em></p>

</details>

---

## **3. Orquestração e Ingestão**

### 🐳 **Docker Compose**
**Papel:** Orquestração de Containers.
* **Função:** Sobe e conecta os serviços do Airflow, Dremio, Nessie e Postgres em uma rede interna isolada, garantindo portabilidade.

### 🌪️ **Apache Airflow**
**Papel:** Orquestrador Geral (O "Maestro").
* **Função:** Gerencia todo o pipeline de dados. Monitora o Google Drive, dispara os workers de ingestão e aciona o **dbt** para as transformações.
* **Otimização:** Configurado com `LocalExecutor` para paralelizar tarefas nos 4 núcleos da CPU.

### 🦆 **DuckDB (Worker)**
**Papel:** Processamento Leve de Ingestão.
* **Função:** Banco analítico embutido (*in-process*) que roda dentro do worker do Airflow.
* **No MVP:** Lê arquivos CSV brutos da camada Landing, limpa e converte para **Parquet** na camada Bronze. Sua eficiência permite processar gigabytes de dados com pouca RAM.

### ☁️ **Google Cloud Storage (GCS)**
**Papel:** Armazenamento de Objetos (Storage Layer).
* **Camadas:**
    * **Landing:** Dados brutos (CSV).
    * **Bronze:** Dados históricos convertidos para Parquet.
    * **Silver/Gold:** Dados de negócio no formato **Apache Iceberg**.

---

## 4. **O Motor do Lakehouse (Engine & Transformação)**

### 💎 **Dremio**
**Papel:** SQL Lakehouse Engine.
* **Função:** Motor de consulta que permite executar SQL diretamente sobre o Data Lake.
* **Performance:** Tunado com **Heap de 2.5GB** e **Direct Memory de 1GB** para operar no limite do hardware, delegando carga excedente para o Swap.

### 🧊 **Apache Iceberg**
**Papel:** Formato de Tabela Aberto.
* **Função:** Camada de abstração que traz transações ACID e *Time Travel* para o Data Lake, permitindo updates e deletes seguros em arquivos Parquet.

### 🛠️ **dbt (data build tool)**
**Papel:** Lógica de Transformação.
* **Função:** Compila as regras de negócio (SQL) e envia para o Dremio executar. Transforma dados da Bronze em tabelas Silver (limpas) e Gold (agregadas).

---

## **5. Governança**

### 🦕 **Project Nessie**
**Papel:** Catálogo de Dados (Git-for-Data).
* **Função:** Gerencia as versões das tabelas Iceberg.
* **Recurso:** Permite criar *Branches* de dados para testes isolados antes de fazer *Merge* na produção, similar ao Git para código.

### O que é a **Arquitetura Moderna de Dados** ?
#### É uma arquitetura **flexível**, **escalável** e **baseada na nuvem**. Ela surge dos modelos tradicionais para suportar o volume e a velocidade dos dados atuais, integrando **Big Data**, **IA** e **governança**. Ela tem como características principais:
- #### **Nuvem** e **Tempo Real**: Prioriza a elasticidade (AWS, Azure, GCP) e a disponibilidade imediata dos dados para tomadas de decisão rápidas. Neste MVP, vamos utilizar a GCP como nuvem principal, mas a solução que vamos implementar é totalmente agnóstica em questões de hospedagem.
- #### **Dados como Produto**: O gerenciamento é focado no valor que o dado gera para o negócio.
- #### **Governança e Segurança**:  Implementadas na base (considerando LGPD), com forte gestão de metadados e interoperabilidade via APIs.
#### As abordagens mais populares dessa arquitetura são:
- #### **Data Lakehouse**: O melhor dos dois mundos (flexibilidade do Lake + confiabilidade do Warehouse). ***Será a arquitetura que utilizaremos nesse MVP.***
- #### **Data Mesh**: Descentraliza a propriedade dos dados por domínios de negócio.
- #### **Modern Data Stack (MDS)**: Orquestração de diversas ferramentas especializadas.

### A **Arquitetura moderna de dados** tem como principal objetivo unificar dados, análises e cargas de trabalho de Inteligência Artificial de forma **eficiente, confiável e escalável**.
### ***Nesse MVP, vamos focar no uso do Data Lakehouse com a Arquitetura Medalhão***
### Antes da seção sobre a infraestrutura que será utilizada, passaremos rapidamente sobre os significados e propósitos do **Data Lakehouse** e **Arquitetura Medalhão**


### **Data Lakehouse**
<center>
<img src="https://drive.google.com/uc?id=1jV69fy2AMAd4YZx0BACaQPwHQLkCJNH1" alt="Girl in a jacket" width="400">
<center />



## **Base de Dados alvo**

### [Aqui, dizer quais as bases de dados serão utilizadas e seu catálogo de dados]

## **Coleta dos dados**

## **Modelagem**

## **Carga**

## **Análise**

### **Qualidade dos dados**

### **Solução do Problema**